In [87]:
from langchain_anthropic import ChatAnthropic

pinecone_namespace = "simple-rag-1"
chat_history = [{
    "role": "human",
    "message": "Describe the hierarchy and organizational structure of the judicial system in Pakistan."
},
    {
    "role": "ai",
    "message": "The judicial system in Pakistan follows a hierarchical structure with the Supreme Court at the top, followed by High Courts in each province, and subordinate courts which include district and sessions courts, civil and judicial magistrates."
},
    {
    "role": "human",
    "message": "what is the role of subordinate courts"
},
    {
    "role": "ai",
    "message": "Subordinate courts handle the bulk of civil and criminal cases at the district level."
}

]
question = "what challenges are faced by them"

model = ChatAnthropic(model="claude-3-sonnet-20240229")

test_response = model.invoke("Hello how are you?")

### Rephrase Query According to Conversation

In [88]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

rephrase_query_prompt = [
    SystemMessage(
        "Given the following conversation, repharase the followup question to be a standalone question, that is understandable and optimized for similarity search while preserving the meaning. The output should just be the rephrased question")
] + [
    HumanMessage(entry["message"]) if entry["role"] == "human" else AIMessage(entry["message"]) for entry in chat_history
] + [HumanMessage(f"followup: \n {question}")]
rephrase_query_answer = model.invoke(rephrase_query_prompt)
query = rephrase_query_answer.content
print(query)

What are the key challenges faced by subordinate courts in the judicial system of Pakistan?


### Find relevant chunks to rephrase query

In [89]:
from utils import get_relevant_chunks

relevant_chunks = get_relevant_chunks(query, pinecone_namespace, 10)
relevant_chunks_text = [item["metadata"]["text"] for item in relevant_chunks]

### Rerank chunks

In [90]:
from utils import rerank_chunks, print_chunks

rerank_results = rerank_chunks(query, relevant_chunks_text)
most_relevant_chunks = rerank_results[0:3]
most_relevant_chunks_text = "----------------------\n".join([c["text"] for c in most_relevant_chunks])
print(most_relevant_chunks_text)

Judicial Statistics of Pakistan, op.cit.
Judicial Statistics of Pakistan, op.cit.
19
The Judicial System of Pakistan
Supreme Court came down from 29 to 18, following a ruling of this Court in the case of Sindh High
Court Bar Association v Federation wherein the judges who had taken oath under the Provisional
Constitution Order (PCO) 2007. Similarly, close to one hundred judges of High Courts were also hit by
the ruling and were denotified. The High Courts then took some time to regain their strength,
through fresh induction.
The Subordinate Courts continue to face the daunting challenge of shortage of strength, support
staff, equipment and other budgetary constraints. The judiciary – 3rd pillar of the State – is not
getting even 1% of the total budgetary allocation of the federal/provincial governments.
Notwithstanding the deficiency in strength and budgetary constraints, due to efficient----------------------
courts have heavy dockets, the Subordinate Courts have to bear the brunt. A

### Prompt LLM for an Actual Answer

In [91]:
get_answer_prompt = [
    SystemMessage("You are a legal assistant, you are given some knowledge about the topic, and a question. You must answer user's question only if its evident from the knowledge, If you cannot find or extract enough knowledge, let the user know that as per your limited knowledge you are not sure and won't be able to answer them"),
    HumanMessage(f"Knowledge is made up of top text chunks in order, acquired after performing a similarity search in the knwoledge base for user's query \n Knowledge: \n {most_relevant_chunks_text} \n question: {query}")
]
get_answer_res = model.invoke(get_answer_prompt)
print(get_answer_res.content)

Based on the provided knowledge, some of the key challenges faced by subordinate courts in the judicial system of Pakistan are:

1. Shortage of strength - There is a shortage of judicial officers and support staff in the subordinate courts.

2. Lack of equipment and infrastructure - The knowledge mentions shortage of courtrooms and office equipment in the subordinate courts.

3. Budgetary constraints - The judiciary is not getting even 1% of the total budgetary allocation from federal/provincial governments, leading to resource crunch.

4. Huge backlog of pending cases - Due to shortage of judges and rise in litigation, there is an accumulation of enormous backlog of pending cases in subordinate courts, leading to delays in deciding cases.

5. Heavy dockets/workload - The knowledge states that subordinate courts have to bear the brunt as around 90% of litigation in Pakistan is conducted at this level, resulting in heavy caseloads for judges.

So in summary, the key challenges highlight